In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
!pip install llama_index.embeddings.huggingface
!pip install llama_index

In [9]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

Settings.embed_model = HuggingFaceEmbedding(
    # model_name="BAAI/bge-small-en-v1.5"
    model_name="BAAI/bge-base-en-v1.5"
)
# Settings.embed_model = OpenAIEmbedding()
# embed_model = OpenAIEmbedding(embed_batch_size=10)
# Settings.embed_model = embed_model
# llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
# Settings.llm = llm

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [11]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

month_mapping = {
    "01": "January",
    "02": "February",
    "03": "March",
    "04": "April",
    "05": "May",
    "06": "June",
    "07": "July",
    "08": "August",
    "09": "September",
    "10": "October",
    "11": "November",
    "12": "December"
}
# Define a function to extract metadata
def extract_month_metadata(file_path):
    # Extract month from file name or content
    month = file_path.split('/')[-1].split('.')[0]  # Assuming file name format like 'statement_01.pdf'
    month_name = month_mapping.get(month, "Invalid month")
    print(month_name)
    return {"month": month_name}

# Load documents with metadata
reader = SimpleDirectoryReader(input_dir="./data/output/2023_sbi", file_metadata=extract_month_metadata)
documents = reader.load_data()

# Create an index with the documents
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist(persist_dir="./indexes")

January
February
March
April
May
June
July
August
September
October
November
December


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
from llama_index.llms.ollama import Ollama

Settings.llm = Ollama(model="llama3.2:3b",
                      temperature=0.1,
                      system_prompt="You are a financial assistant, you help with analyzing the bank transactions\
                      . If you are not sure about transaction detail, then you do not give any output and you should say \"I cannot aprovide that information\"",
                      request_timeout=300.0)

In [15]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
    verbose=True,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="compact_accumulate",
    verbose=True,
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [18]:
# query
response = query_engine.query("Can you give list of categories of transactions for month of March?")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
Response 1: Debit, Credit, Transfer
